# Process Bacteria

... description of the notebook goes here ...

### Prerequisites:
- conda (Miniconda or Anaconda): https://conda.io/projects/conda/en/latest/user-guide/install/index.html
- scikit-bio: `conda install -c https://conda.anaconda.org/biocore scikit-bio`
- prophage tools & their setup (follow instructions in section 2.)

## 1. Notebook Setup

Import the necessary libraries and set the data & output paths

In [2]:
import os
import shutil
import subprocess
from pathlib import Path

import skbio

In [3]:
# Path to the folder containing genomic data (relative to current working directory)
data_dir = Path(Path.cwd(), '..', 'data')
data_dir.mkdir(exist_ok=True)
(data_dir / 'compressed').mkdir(exist_ok=True)

# Path to the output folder
out_dir = Path(Path.cwd(), '..', 'out')
out_dir.mkdir(exist_ok=True)

## 2. Prophage Tool Setup

Select prophage CLI tools you intend to use. The recommended ones include:

 - PhageBoost: https://github.com/ku-cbd/PhageBoost
 - PhiSpy: https://github.com/linsalrob/PhiSpy

For each tool:

 1. Create new conda enviroment: `conda create -n [name_of_the_tool]`
 1. Install the tool and its prerequisites according to author's instructions
 1. Register the path to the binary and arguments the tool should be invoked with using the function below:

In [4]:
tools = []

def register_tool(tool_name, tool_path, input_arg, output_arg, tool_input_type='fasta', separate_subsequences=False, **cli_args):
    def invoke_tool(inp, out):
        return f'{tool_path} {input_arg} {inp} {output_arg} {out} {" ".join([f"--{f} {a}" for f, a in cli_args.items()])}'
    tools.append({
        'name': tool_name,
        'input_type': tool_input_type,
        'separate_subsequences': separate_subsequences,
        'cli_call': invoke_tool
    })

# Register tools here
register_tool('PhageBoost',
    tool_path='~/miniconda3/envs/phage-boost/bin/PhageBoost',
    input_arg='-f',
    output_arg='-o',
    # Other args go here
)

register_tool('PhiSpy',
    tool_input_type='gb',
    tool_path='~/miniconda3/envs/phispy/bin/PhiSpy.py',
    input_arg='',
    output_arg='-o',
    # Other args go here
)

## 3. Data extraction

Download genomic data for the bacteria of interest from [NCBI Assembly](https://www.ncbi.nlm.nih.gov/assembly) in fasta (.fna) and genbank (.gbff) formats. Name them "[taxid]_(fasta|gb).tar" and put them in "compressed" folder in your data folder, then run the following code:

In [5]:
# Change to True to extract data anew
overwrite_data = False

# 1.c decompress all downloaded assemblies
for entry in os.scandir(data_dir / 'compressed'):
    if entry.is_file():
        taxid, file_type = entry.name.split('.')[0].split('_')
        file_path = data_dir / taxid / file_type

        if file_path.exists():
            if overwrite_data:
                shutil.rmtree(file_path)
            else:
                continue

        file_path.mkdir(exist_ok=True, parents=True)

        # Run tar command in a subprocess to extract the archive
        subprocess.run(['tar', '-xf', entry.path, '-C', file_path, '--strip-components', '1'])

        # Unzip all extracted files
        for f in os.scandir(file_path):
            if (f.path.endswith('.gz')):
               subprocess.run(['gzip', '-d', f.path])
                # Rename the files to include only assembly accession id
               f_path = Path(f.path.replace('.gz', ''))
               f_path.rename(f_path.parents[0] / ('_'.join(f_path.stem.split('_')[:2]) + f_path.suffix))
            else:
                # Remove files that don't contain genomic sequences
                os.remove(f.path)

### 3.1 Flatten FASTA files

Many tools work better with only one sequence per fasta file, so we need to split the files with multiple sequences into individual ones:

In [6]:
for entry in os.scandir(data_dir):
    fasta_dir = Path(entry.path, 'fasta')
    if entry.is_dir() and fasta_dir.exists():
        for f in os.scandir(fasta_dir):
            if not f.is_file():
                continue
            
            f_path = Path(f.path)
            dir_path = (f_path.parents[0] / f_path.stem)
            dir_path.mkdir(exist_ok=True)

            for seq in skbio.io.read(str(f_path), format='fasta'):
                seq.write(str(dir_path / (seq.metadata['id'] + '.fasta')), format='fasta')
            
            os.remove(f_path)

## 4. Run the tools

Predict the prophages from the data sequences and save the results in \[output_folder\]/\[tool_name\]/\[taxid\]/\[assembly_accession\]/:

In [7]:
# Change to True to process data anew
overwrite_output = True

for tool in tools:
    for entry in os.scandir(data_dir):
        # Use only data in the correct format for the tool
        seq_dir = Path(entry.path, tool['input_type'])
        if entry.is_dir() and seq_dir.exists():
            for assembly in os.scandir(seq_dir):
                print(assembly.path)
                print('\n')

                # Create output path
                tool_out_path = out_dir / tool['name'] / entry.name / '.'.join(assembly.name.split('.')[:2])
                if tool_out_path.exists():
                    if overwrite_output or not (tool_out_path / '.done').exists():
                        shutil.rmtree(tool_out_path)
                    else:
                        print('\n--------------------------------------------------\n')
                        continue

                tool_out_path.mkdir(exist_ok=True, parents=True)

                # Run the tool
                if assembly.is_file():
                    !{tool['cli_call'](assembly.path, str(tool_out_path))}
                elif assembly.is_dir():
                    for f in os.scandir(assembly.path):
                        !{tool['cli_call'](f.path, str(tool_out_path / '.'.join(f.name.split('.')[:-1])) if tool['separate_subsequences'] else str(tool_out_path))}

                # Mark as done
                (tool_out_path / '.done').touch()
                print('\n--------------------------------------------------\n')

ing_model.html
time after predictions: 2.2317392826080322
{}
no phages found 
processing: WOUN01000007
time after genecalls: 0.13173556327819824
time after feature calculations: 0.4884936809539795
[11:51:49] WARNING: ../src/tree/./updater_quantile_hist.h:135: Attempted to load internal configuration for a model file that was generated by a previous version of XGBoost. A likely cause for this warning is that the model was saved with saveRDS() in R or pickle.dump() in Python. We strongly ADVISE AGAINST using saveRDS() or pickle.dump() so that the model remains accessible in current and upcoming XGBoost releases. Please use xgb.save() instead to preserve models for the long term. For more details and explanation, see https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html
time after predictions: 2.0665030479431152
{}
no phages found 
processing: WOUN01000001
time after genecalls: 1.5512664318084717
time after feature calculations: 4.097052574157715
[11:51:56] WARNING: ../src/